In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization

Using TensorFlow backend.


In [32]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)
train_df_y.reset_index()
df = pd.concat([train_df,test_df])
df['datetime'] = pd.to_datetime(df.datetime)
#何月か
df['Month'] = df['datetime'].dt.month
#何日か
df['DayofMonth'] = df['datetime'].dt.day
#何曜日か
df['dayofweek'] = df['datetime'].dt.dayofweek
#何年か
df['year'] = df['datetime'].dt.year
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df["year"])

df["year_num"] = le.transform(df["year"])
#ダミー変数化
#何月か
month_df = pd.get_dummies(df['Month'], prefix='M', prefix_sep='_')
#何日か
dayofmonth_df = pd.get_dummies(df["DayofMonth"], prefix='DM', prefix_sep='_')
#何曜日か
dayofweek_df = pd.get_dummies(df['dayofweek'], prefix='DW', prefix_sep='_')
#何年か
year_df= pd.get_dummies(df['year'], prefix='y', prefix_sep='_')
#priceはどうか 
price_am_df = pd.get_dummies(df['price_am'], prefix='PA', prefix_sep='_')
price_pm_df = pd.get_dummies(df['price_pm'], prefix='PP', prefix_sep='_')
#datetimeを保存
df_id = df[['datetime']]
#year_df削除
df = pd.concat([df,
                month_df,
                dayofmonth_df,
                dayofweek_df,
               price_am_df,
               price_pm_df,
               year_df], axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]
test_df_datetime = test_df["datetime"]

In [26]:
# df = pd.concat([df,
#                 month_df,
#                 dayofmonth_df,
#                 dayofweek_df,
#                price_am_df,
#                price_pm_df,
#                year_df], axis=1)

# train_df = df[0:2101].reset_index()
# train_df = pd.concat([train_df,train_df_y],axis=1)
# train_df = train_df.drop("index",axis=1)
# test_df = df[2101:]

In [33]:
test_df_y_2017= test_df["y_2017"]
df = df.drop(['price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num","year",'y_2017'],axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [34]:
df.columns

Index(['datetime', 'client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3',
       'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11',
       'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19',
       'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27',
       'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3',
       'DW_4', 'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4',
       'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5',
       'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016'],
      dtype='object')

In [42]:
train_df.columns

Index(['datetime', 'client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3',
       'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11',
       'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19',
       'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27',
       'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3',
       'DW_4', 'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4',
       'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5',
       'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016',
       'y'],
      dtype='object')

In [ ]:
train_df

In [39]:
x = train_df.drop(["y","datetime"],axis=1)
y = train_df["y"]

In [56]:
model = Sequential()
model.add(Dense(140,input_dim=73,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(70,input_dim=73,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.05,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model.fit(x.values,y.values,nb_epoch=1000)

Epoch 1/1000
2101/2101 [==============================] - 0s - loss: 9.6506 - mean_absolute_error: 9.6506       
Epoch 2/1000
2101/2101 [==============================] - 0s - loss: 7.1837 - mean_absolute_error: 7.1837     
Epoch 3/1000
2101/2101 [==============================] - 0s - loss: 6.8575 - mean_absolute_error: 6.8575     
Epoch 4/1000
2101/2101 [==============================] - 0s - loss: 6.4368 - mean_absolute_error: 6.4368     
Epoch 5/1000
2101/2101 [==============================] - 0s - loss: 6.5439 - mean_absolute_error: 6.5439     
Epoch 6/1000
2101/2101 [==============================] - 0s - loss: 6.4920 - mean_absolute_error: 6.4920     
Epoch 7/1000
2101/2101 [==============================] - 0s - loss: 6.4896 - mean_absolute_error: 6.4896     
Epoch 8/1000
2101/2101 [==============================] - 0s - loss: 6.4256 - mean_absolute_error: 6.4256     
Epoch 9/1000
2101/2101 [==============================] - 0s - loss: 6.3432 - mean_absolute_error: 6.3432     

2101/2101 [==============================] - 0s - loss: 1.6000 - mean_absolute_error: 1.6000     
Epoch 146/1000
2101/2101 [==============================] - 0s - loss: 1.5068 - mean_absolute_error: 1.5068     
Epoch 147/1000
2101/2101 [==============================] - 0s - loss: 1.6259 - mean_absolute_error: 1.6259     
Epoch 148/1000
2101/2101 [==============================] - 0s - loss: 1.6005 - mean_absolute_error: 1.6005     
Epoch 149/1000
2101/2101 [==============================] - 0s - loss: 1.4651 - mean_absolute_error: 1.4651     
Epoch 150/1000
2101/2101 [==============================] - 0s - loss: 1.5315 - mean_absolute_error: 1.5315     
Epoch 151/1000
2101/2101 [==============================] - 0s - loss: 1.4549 - mean_absolute_error: 1.4549     
Epoch 152/1000
2101/2101 [==============================] - 0s - loss: 1.4424 - mean_absolute_error: 1.4424     
Epoch 153/1000
2101/2101 [==============================] - 0s - loss: 1.4519 - mean_absolute_error: 1.4519    

2101/2101 [==============================] - 0s - loss: 0.7527 - mean_absolute_error: 0.7527     
Epoch 289/1000
2101/2101 [==============================] - 0s - loss: 0.7098 - mean_absolute_error: 0.7098     
Epoch 290/1000
2101/2101 [==============================] - 0s - loss: 0.6947 - mean_absolute_error: 0.6947     
Epoch 291/1000
2101/2101 [==============================] - 0s - loss: 0.7984 - mean_absolute_error: 0.7984     
Epoch 292/1000
2101/2101 [==============================] - 0s - loss: 0.7383 - mean_absolute_error: 0.7383     
Epoch 293/1000
2101/2101 [==============================] - 0s - loss: 0.7708 - mean_absolute_error: 0.7708     
Epoch 294/1000
2101/2101 [==============================] - 0s - loss: 0.7344 - mean_absolute_error: 0.7344     
Epoch 295/1000
2101/2101 [==============================] - 0s - loss: 0.7128 - mean_absolute_error: 0.7128     
Epoch 296/1000
2101/2101 [==============================] - 0s - loss: 0.7267 - mean_absolute_error: 0.7267    

2101/2101 [==============================] - 0s - loss: 0.5059 - mean_absolute_error: 0.5059     
Epoch 433/1000
2101/2101 [==============================] - 0s - loss: 0.4760 - mean_absolute_error: 0.4760     
Epoch 434/1000
2101/2101 [==============================] - 0s - loss: 0.4689 - mean_absolute_error: 0.4689     
Epoch 435/1000
2101/2101 [==============================] - 0s - loss: 0.4803 - mean_absolute_error: 0.4803     
Epoch 436/1000
2101/2101 [==============================] - 0s - loss: 0.5050 - mean_absolute_error: 0.5050     
Epoch 437/1000
2101/2101 [==============================] - 0s - loss: 0.4943 - mean_absolute_error: 0.4943     
Epoch 438/1000
2101/2101 [==============================] - 0s - loss: 0.4382 - mean_absolute_error: 0.4382     
Epoch 439/1000
2101/2101 [==============================] - 0s - loss: 0.4218 - mean_absolute_error: 0.4218     
Epoch 440/1000
2101/2101 [==============================] - 0s - loss: 0.5015 - mean_absolute_error: 0.5015    

2101/2101 [==============================] - 0s - loss: 0.3490 - mean_absolute_error: 0.3490     
Epoch 577/1000
2101/2101 [==============================] - 0s - loss: 0.3674 - mean_absolute_error: 0.3674     
Epoch 578/1000
2101/2101 [==============================] - 0s - loss: 0.3933 - mean_absolute_error: 0.3933     
Epoch 579/1000
2101/2101 [==============================] - 0s - loss: 0.3477 - mean_absolute_error: 0.3477     
Epoch 580/1000
2101/2101 [==============================] - 0s - loss: 0.3901 - mean_absolute_error: 0.3901     
Epoch 581/1000
2101/2101 [==============================] - 0s - loss: 0.3514 - mean_absolute_error: 0.3514     
Epoch 582/1000
2101/2101 [==============================] - 0s - loss: 0.3506 - mean_absolute_error: 0.3506     
Epoch 583/1000
2101/2101 [==============================] - 0s - loss: 0.3628 - mean_absolute_error: 0.3628     
Epoch 584/1000
2101/2101 [==============================] - 0s - loss: 0.3249 - mean_absolute_error: 0.3249    

2101/2101 [==============================] - 0s - loss: 0.2504 - mean_absolute_error: 0.2504     
Epoch 721/1000
2101/2101 [==============================] - 0s - loss: 0.2561 - mean_absolute_error: 0.2561     
Epoch 722/1000
2101/2101 [==============================] - 0s - loss: 0.2861 - mean_absolute_error: 0.2861     
Epoch 723/1000
2101/2101 [==============================] - 0s - loss: 0.2706 - mean_absolute_error: 0.2706     
Epoch 724/1000
2101/2101 [==============================] - 0s - loss: 0.3153 - mean_absolute_error: 0.3153     
Epoch 725/1000
2101/2101 [==============================] - 0s - loss: 0.2507 - mean_absolute_error: 0.2507     
Epoch 726/1000
2101/2101 [==============================] - 0s - loss: 0.2831 - mean_absolute_error: 0.2831     
Epoch 727/1000
2101/2101 [==============================] - 0s - loss: 0.2893 - mean_absolute_error: 0.2893     
Epoch 728/1000
2101/2101 [==============================] - 0s - loss: 0.2967 - mean_absolute_error: 0.2967    

2101/2101 [==============================] - 0s - loss: 0.2287 - mean_absolute_error: 0.2287     
Epoch 865/1000
2101/2101 [==============================] - 0s - loss: 0.2608 - mean_absolute_error: 0.2608     
Epoch 866/1000
2101/2101 [==============================] - 0s - loss: 0.2231 - mean_absolute_error: 0.2231     
Epoch 867/1000
2101/2101 [==============================] - 0s - loss: 0.2320 - mean_absolute_error: 0.2320     
Epoch 868/1000
2101/2101 [==============================] - 0s - loss: 0.2294 - mean_absolute_error: 0.2294     
Epoch 869/1000
2101/2101 [==============================] - 0s - loss: 0.2127 - mean_absolute_error: 0.2127     
Epoch 870/1000
2101/2101 [==============================] - 0s - loss: 0.2116 - mean_absolute_error: 0.2116     
Epoch 871/1000
2101/2101 [==============================] - 0s - loss: 0.2283 - mean_absolute_error: 0.2283     
Epoch 872/1000
2101/2101 [==============================] - 0s - loss: 0.2306 - mean_absolute_error: 0.2306    

In [57]:
model.predict(x.values)

array([[  17.22574234],
       [  18.19899559],
       [  19.88523102],
       ..., 
       [  95.9719696 ],
       [  98.71368408],
       [ 103.92834473]], dtype=float32)

In [59]:
x.columns

Index(['client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7',
       'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3', 'DM_4',
       'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11', 'DM_12',
       'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19', 'DM_20',
       'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27', 'DM_28',
       'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3', 'DW_4',
       'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4', 'PA_5',
       'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5', 'y_2010',
       'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016'],
      dtype='object')

In [48]:
pred_df= test_df

In [50]:
pred_df.columns

Index(['datetime', 'client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3',
       'DM_4', 'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11',
       'DM_12', 'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19',
       'DM_20', 'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27',
       'DM_28', 'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3',
       'DW_4', 'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4',
       'PA_5', 'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5',
       'y_2010', 'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016'],
      dtype='object')

In [53]:
pred_df = pred_df.drop(["datetime"],axis=1)

In [47]:
pred_df = pd.concat([pred_df,test_df_y_2017],axis=1)

In [54]:
pred_df["y_2016"] = int(1)
pred_df["y"] = model.predict(pred_df.values)
pred_df = pd.concat([pred_df,test_df_y_2017],axis=1)
pred_df["y"] = (1 - pred_df['close']) * pred_df.y + (1 - pred_df['close'])*(pred_df["y_2017"])*6
submission = pd.concat([test_df_datetime, pred_df.y], axis=1)
submission.to_csv("submission_1030.csv", index=False,header=False)

In [ ]:
pred_df["y"] = model.predict(test_df.values)
pred_df["y"] = (1 - pred_df['close']) * pred_df.y + (1 - pred_df['close'])*(pred_df["y_2017"])*6 
submission = pd.concat([test_df_datetime, pred_df.y], axis=1)
submission.to_csv("submission_1030.csv", index=False,header=False)

In [55]:
submission
#9.24662

,datetime,y
0,2016-04-01,70.506294
1,2016-04-02,57.582363
2,2016-04-03,85.662636
3,2016-04-04,58.572403
4,2016-04-05,52.187897
5,2016-04-06,49.476460
6,2016-04-07,61.398998
7,2016-04-08,53.038548
8,2016-04-09,68.831276
9,2016-04-10,75.769897


In [ ]:
#y_2017の処理をする
test_df = pd.concat([test_df,test_df_y_2017],axis=1)
test_df["y"] = model.predict(test_df.values)
test_df["y"] = (1 - test_df['close']) * test_df.y + (1 - test_df['close'])*(test_df["y_2017"])*6 
submission = pd.concat([test_df_datetime, test_df.y], axis=1)
submission.to_csv("submission.csv", index=False,header=False)

# ニューラルネットワーク

# 使った特徴量
'client', 'close', 'M_1', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6', 'M_7',
       'M_8', 'M_9', 'M_10', 'M_11', 'M_12', 'DM_1', 'DM_2', 'DM_3', 'DM_4',
       'DM_5', 'DM_6', 'DM_7', 'DM_8', 'DM_9', 'DM_10', 'DM_11', 'DM_12',
       'DM_13', 'DM_14', 'DM_15', 'DM_16', 'DM_17', 'DM_18', 'DM_19', 'DM_20',
       'DM_21', 'DM_22', 'DM_23', 'DM_24', 'DM_25', 'DM_26', 'DM_27', 'DM_28',
       'DM_29', 'DM_30', 'DM_31', 'DW_0', 'DW_1', 'DW_2', 'DW_3', 'DW_4',
       'DW_5', 'DW_6', 'PA_-1', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_4', 'PA_5',
       'PP_-1', 'PP_0', 'PP_1', 'PP_2', 'PP_3', 'PP_4', 'PP_5', 'y_2010',
       'y_2011', 'y_2012', 'y_2013', 'y_2014', 'y_2015', 'y_2016'

## 学習させる際の処理
- traindataの全データを学習
- LeakyRelu
- predictさせる際の処理　test_dfのy_2016を1にして予測

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization

In [ ]:
model = Sequential()
model.add(Dense(140,input_dim=73,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(70,input_dim=73,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.05,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model.fit(x.values,y.values,nb_epoch=1000)

In [60]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)
train_df_y.reset_index()
df = pd.concat([train_df,test_df])
df['datetime'] = pd.to_datetime(df.datetime)
#何月か
df['Month'] = df['datetime'].dt.month
#何日か
df['DayofMonth'] = df['datetime'].dt.day
#何曜日か
df['dayofweek'] = df['datetime'].dt.dayofweek
#何年か
df['year'] = df['datetime'].dt.year
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [62]:
train_df.columns

Index(['datetime', 'client', 'close', 'price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek', 'year', 'y'],
      dtype='object')

In [78]:
import xgboost as xgb
#モデルのインスタンス作成
#5 300
mod = xgb.XGBRegressor(max_depth =10,n_estimators=500)
X = train_df.drop(["datetime","y"], axis=1)
 

Y = train_df["y"]
#欠損値のデータを学習させずにモデル生成
mod.fit(X,Y,eval_metric=['mae'])

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [81]:
Y.shape

(2101,)

In [71]:
X.columns

Index(['client', 'close', 'price_am', 'price_pm', 'Month', 'DayofMonth',
       'dayofweek', 'year'],
      dtype='object')

In [79]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y,mod.predict(X))

0.0036627955802107696

In [68]:
def logregobj(dtrain, preds):
    grad = preds
    hess = preds * (1.0-preds)
    return grad, hess

model = xgb.XGBRegressor(objective=fair_obj)
model.fit(X,Y)

AttributeError: 'numpy.ndarray' object has no attribute 'get_labels'

In [67]:
def fair_obj(preds, dtrain):
    """y = c * abs(x) - c * np.log(abs(abs(x) + c))"""
    x = dtrain.get_labels() - preds
    c = 1
    den = abs(x) + c
    grad = c*x / den
    hess = c*c / den ** 2
    return grad, hess

In [ ]:
x = df_h.drop(["y","datetime"],axis=1)
y = df_h["y"]

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df_y = train_df[["y"]]
train_df = train_df.drop(["y"],axis=1)
train_df_y.reset_index()
df = pd.concat([train_df,test_df])
df['datetime'] = pd.to_datetime(df.datetime)
#何月か
df['Month'] = df['datetime'].dt.month
#何日か
df['DayofMonth'] = df['datetime'].dt.day
#何曜日か
df['dayofweek'] = df['datetime'].dt.dayofweek
#何年か
df['year'] = df['datetime'].dt.year
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df["year"])

df["year_num"] = le.transform(df["year"])
#ダミー変数化
#何月か
month_df = pd.get_dummies(df['Month'], prefix='M', prefix_sep='_')
#何日か
dayofmonth_df = pd.get_dummies(df["DayofMonth"], prefix='DM', prefix_sep='_')
#何曜日か
dayofweek_df = pd.get_dummies(df['dayofweek'], prefix='DW', prefix_sep='_')
#何年か
year_df= pd.get_dummies(df['year'], prefix='y', prefix_sep='_')
#priceはどうか 
price_am_df = pd.get_dummies(df['price_am'], prefix='PA', prefix_sep='_')
price_pm_df = pd.get_dummies(df['price_pm'], prefix='PP', prefix_sep='_')
#datetimeを保存
df_id = df[['datetime']]
#year_df削除
df = pd.concat([df,
                month_df,
                dayofmonth_df,
                dayofweek_df,
               price_am_df,
               price_pm_df], axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]
test_df_datetime = test_df["datetime"]

In [3]:

df = df.drop(['price_am', 'price_pm', 'Month',
       'DayofMonth', 'dayofweek',"year_num","year",],axis=1)
train_df = df[0:2101].reset_index()
train_df = pd.concat([train_df,train_df_y],axis=1)
train_df = train_df.drop("index",axis=1)
test_df = df[2101:]

In [4]:
train_df = train_df[train_df["close"]==0]
df_h = train_df[train_df["datetime"]>="2015-03-30"]


In [5]:
x = df_h.drop(["y","datetime"],axis=1)
y = df_h["y"]

In [27]:
df_h_t = test_df.drop(['datetime'],axis=1)


In [7]:
x

,client,close,M_1,M_2,M_3,M_4,M_5,M_6,M_7,M_8,...,PA_3,PA_4,PA_5,PP_-1,PP_0,PP_1,PP_2,PP_3,PP_4,PP_5
1733,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1734,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1735,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1736,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1737,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1738,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1739,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1740,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1741,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1742,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [26]:
model = Sequential()
model.add(Dense(365,input_dim=66,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(66,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))
model.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.04,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model.fit(x.values,y.values,nb_epoch=1000)

Epoch 1/1000
363/363 [==============================] - 0s - loss: 26.9018 - mean_absolute_error: 26.9018     
Epoch 2/1000
363/363 [==============================] - 0s - loss: 12.9205 - mean_absolute_error: 12.9205     
Epoch 3/1000
363/363 [==============================] - 0s - loss: 11.2088 - mean_absolute_error: 11.2088     
Epoch 4/1000
363/363 [==============================] - 0s - loss: 9.2925 - mean_absolute_error: 9.2925     
Epoch 5/1000
363/363 [==============================] - 0s - loss: 9.2614 - mean_absolute_error: 9.2614     
Epoch 6/1000
363/363 [==============================] - 0s - loss: 8.4364 - mean_absolute_error: 8.4364     
Epoch 7/1000
363/363 [==============================] - 0s - loss: 7.9502 - mean_absolute_error: 7.9502     
Epoch 8/1000
363/363 [==============================] - 0s - loss: 8.3672 - mean_absolute_error: 8.3672     
Epoch 9/1000
363/363 [==============================] - 0s - loss: 7.9118 - mean_absolute_error: 7.9118     
Epoch 10/1000

363/363 [==============================] - 0s - loss: 3.0588 - mean_absolute_error: 3.0588     
Epoch 76/1000
363/363 [==============================] - 0s - loss: 2.6661 - mean_absolute_error: 2.6661     
Epoch 77/1000
363/363 [==============================] - 0s - loss: 2.1851 - mean_absolute_error: 2.1851     
Epoch 78/1000
363/363 [==============================] - 0s - loss: 2.7586 - mean_absolute_error: 2.7586     
Epoch 79/1000
363/363 [==============================] - 0s - loss: 3.8979 - mean_absolute_error: 3.8979     
Epoch 80/1000
363/363 [==============================] - 0s - loss: 3.0041 - mean_absolute_error: 3.0041     
Epoch 81/1000
363/363 [==============================] - 0s - loss: 2.5453 - mean_absolute_error: 2.5453     
Epoch 82/1000
363/363 [==============================] - 0s - loss: 2.7379 - mean_absolute_error: 2.7379     
Epoch 83/1000
363/363 [==============================] - 0s - loss: 2.8511 - mean_absolute_error: 2.8511     
Epoch 84/1000
363/363 [=

363/363 [==============================] - 0s - loss: 1.4380 - mean_absolute_error: 1.4380     
Epoch 150/1000
363/363 [==============================] - 0s - loss: 1.6344 - mean_absolute_error: 1.6344     
Epoch 151/1000
363/363 [==============================] - 0s - loss: 1.6319 - mean_absolute_error: 1.6319     
Epoch 152/1000
363/363 [==============================] - 0s - loss: 2.0542 - mean_absolute_error: 2.0542     
Epoch 153/1000
363/363 [==============================] - 0s - loss: 1.6802 - mean_absolute_error: 1.6802     
Epoch 154/1000
363/363 [==============================] - 0s - loss: 1.7542 - mean_absolute_error: 1.7542     
Epoch 155/1000
363/363 [==============================] - 0s - loss: 1.4096 - mean_absolute_error: 1.4096     
Epoch 156/1000
363/363 [==============================] - 0s - loss: 1.8923 - mean_absolute_error: 1.8923     
Epoch 157/1000
363/363 [==============================] - 0s - loss: 1.6050 - mean_absolute_error: 1.6050     
Epoch 158/1000
3

363/363 [==============================] - 0s - loss: 1.2200 - mean_absolute_error: 1.2200     
Epoch 223/1000
363/363 [==============================] - 0s - loss: 1.1502 - mean_absolute_error: 1.1502     
Epoch 224/1000
363/363 [==============================] - 0s - loss: 1.2934 - mean_absolute_error: 1.2934     
Epoch 225/1000
363/363 [==============================] - 0s - loss: 1.0672 - mean_absolute_error: 1.0672     
Epoch 226/1000
363/363 [==============================] - 0s - loss: 1.3071 - mean_absolute_error: 1.3071     
Epoch 227/1000
363/363 [==============================] - 0s - loss: 1.0464 - mean_absolute_error: 1.0464     
Epoch 228/1000
363/363 [==============================] - 0s - loss: 1.0995 - mean_absolute_error: 1.0995     
Epoch 229/1000
363/363 [==============================] - 0s - loss: 1.0191 - mean_absolute_error: 1.0191     
Epoch 230/1000
363/363 [==============================] - 0s - loss: 1.0046 - mean_absolute_error: 1.0046     
Epoch 231/1000
3

363/363 [==============================] - 0s - loss: 0.9324 - mean_absolute_error: 0.9324     
Epoch 296/1000
363/363 [==============================] - 0s - loss: 1.3457 - mean_absolute_error: 1.3457     
Epoch 297/1000
363/363 [==============================] - 0s - loss: 1.1581 - mean_absolute_error: 1.1581     
Epoch 298/1000
363/363 [==============================] - 0s - loss: 1.4534 - mean_absolute_error: 1.4534     
Epoch 299/1000
363/363 [==============================] - 0s - loss: 1.3778 - mean_absolute_error: 1.3778     
Epoch 300/1000
363/363 [==============================] - 0s - loss: 1.1935 - mean_absolute_error: 1.1935     
Epoch 301/1000
363/363 [==============================] - 0s - loss: 0.9699 - mean_absolute_error: 0.9699     
Epoch 302/1000
363/363 [==============================] - 0s - loss: 0.8528 - mean_absolute_error: 0.8528     
Epoch 303/1000
363/363 [==============================] - 0s - loss: 0.8889 - mean_absolute_error: 0.8889     
Epoch 304/1000
3

...

363/363 [==============================] - 0s - loss: 0.6614 - mean_absolute_error: 0.6614     
Epoch 661/1000
363/363 [==============================] - 0s - loss: 0.7249 - mean_absolute_error: 0.7249     
Epoch 662/1000
363/363 [==============================] - 0s - loss: 0.6238 - mean_absolute_error: 0.6238     
Epoch 663/1000
363/363 [==============================] - 0s - loss: 0.4732 - mean_absolute_error: 0.4732     
Epoch 664/1000
363/363 [==============================] - 0s - loss: 0.5975 - mean_absolute_error: 0.5975     
Epoch 665/1000
363/363 [==============================] - 0s - loss: 0.4883 - mean_absolute_error: 0.4883     
Epoch 666/1000
363/363 [==============================] - 0s - loss: 0.5591 - mean_absolute_error: 0.5591     
Epoch 667/1000
363/363 [==============================] - 0s - loss: 0.6602 - mean_absolute_error: 0.6602     
Epoch 668/1000
363/363 [==============================] - 0s - loss: 0.6391 - mean_absolute_error: 0.6391     
Epoch 669/1000
3

363/363 [==============================] - 0s - loss: 0.4950 - mean_absolute_error: 0.4950     
Epoch 734/1000
363/363 [==============================] - 0s - loss: 0.4886 - mean_absolute_error: 0.4886     
Epoch 735/1000
363/363 [==============================] - 0s - loss: 0.5217 - mean_absolute_error: 0.5217     
Epoch 736/1000
363/363 [==============================] - 0s - loss: 0.4551 - mean_absolute_error: 0.4551     
Epoch 737/1000
363/363 [==============================] - 0s - loss: 0.6367 - mean_absolute_error: 0.6367     
Epoch 738/1000
363/363 [==============================] - 0s - loss: 0.5392 - mean_absolute_error: 0.5392     
Epoch 739/1000
363/363 [==============================] - 0s - loss: 0.5260 - mean_absolute_error: 0.5260     
Epoch 740/1000
363/363 [==============================] - 0s - loss: 0.4540 - mean_absolute_error: 0.4540     
Epoch 741/1000
363/363 [==============================] - 0s - loss: 0.3932 - mean_absolute_error: 0.3932     
Epoch 742/1000
3

363/363 [==============================] - 0s - loss: 0.4559 - mean_absolute_error: 0.4559     
Epoch 807/1000
363/363 [==============================] - 0s - loss: 0.4099 - mean_absolute_error: 0.4099     
Epoch 808/1000
363/363 [==============================] - 0s - loss: 0.4248 - mean_absolute_error: 0.4248     
Epoch 809/1000
363/363 [==============================] - 0s - loss: 0.3553 - mean_absolute_error: 0.3553     
Epoch 810/1000
363/363 [==============================] - 0s - loss: 0.3963 - mean_absolute_error: 0.3963     
Epoch 811/1000
363/363 [==============================] - 0s - loss: 0.4477 - mean_absolute_error: 0.4477     
Epoch 812/1000
363/363 [==============================] - 0s - loss: 0.4705 - mean_absolute_error: 0.4705     
Epoch 813/1000
363/363 [==============================] - 0s - loss: 0.3566 - mean_absolute_error: 0.3566     
Epoch 814/1000
363/363 [==============================] - 0s - loss: 0.3229 - mean_absolute_error: 0.3229     
Epoch 815/1000
3

363/363 [==============================] - 0s - loss: 0.3589 - mean_absolute_error: 0.3589     
Epoch 880/1000
363/363 [==============================] - 0s - loss: 0.4520 - mean_absolute_error: 0.4520     
Epoch 881/1000
363/363 [==============================] - 0s - loss: 0.4113 - mean_absolute_error: 0.4113     
Epoch 882/1000
363/363 [==============================] - 0s - loss: 0.3727 - mean_absolute_error: 0.3727     
Epoch 883/1000
363/363 [==============================] - 0s - loss: 0.4434 - mean_absolute_error: 0.4434     
Epoch 884/1000
363/363 [==============================] - 0s - loss: 0.3973 - mean_absolute_error: 0.3973     
Epoch 885/1000
363/363 [==============================] - 0s - loss: 0.4414 - mean_absolute_error: 0.4414     
Epoch 886/1000
363/363 [==============================] - 0s - loss: 0.5493 - mean_absolute_error: 0.5493     
Epoch 887/1000
363/363 [==============================] - 0s - loss: 0.5871 - mean_absolute_error: 0.5871     
Epoch 888/1000
3

363/363 [==============================] - 0s - loss: 0.3280 - mean_absolute_error: 0.3280     
Epoch 953/1000
363/363 [==============================] - 0s - loss: 0.3098 - mean_absolute_error: 0.3098     
Epoch 954/1000
363/363 [==============================] - 0s - loss: 0.3515 - mean_absolute_error: 0.3515     
Epoch 955/1000
363/363 [==============================] - 0s - loss: 0.3237 - mean_absolute_error: 0.3237     
Epoch 956/1000
363/363 [==============================] - 0s - loss: 0.3484 - mean_absolute_error: 0.3484     
Epoch 957/1000
363/363 [==============================] - 0s - loss: 0.3618 - mean_absolute_error: 0.3618     
Epoch 958/1000
363/363 [==============================] - 0s - loss: 0.3674 - mean_absolute_error: 0.3674     
Epoch 959/1000
363/363 [==============================] - 0s - loss: 0.3739 - mean_absolute_error: 0.3739     
Epoch 960/1000
363/363 [==============================] - 0s - loss: 0.3594 - mean_absolute_error: 0.3594     
Epoch 961/1000
3

In [28]:
pred_df = df_h_t

pred_df["y"] = model.predict(pred_df.values)
pred_df["y"] = (1 - pred_df['close']) * (pred_df.y + 6)
submission = pd.concat([test_df_datetime, pred_df.y], axis=1)
submission.to_csv("submission_1031_4.csv", index=False,header=False)

In [29]:
submission

,datetime,y
0,2016-04-01,70.004715
1,2016-04-02,87.399681
2,2016-04-03,69.365417
3,2016-04-04,74.429993
4,2016-04-05,65.575272
5,2016-04-06,57.690292
6,2016-04-07,58.807880
7,2016-04-08,58.060955
8,2016-04-09,66.650589
9,2016-04-10,71.034943


0.0

In [35]:
model1 = Sequential()
model1.add(Dense(132,input_dim=66,activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.3)))

model1.add(Dense(1))
opt = keras.optimizers.Adam(lr = 0.04,beta_1=0.9,beta_2=0.999,epsilon=1e-08,decay=0.0003)
model1.compile(optimizer=opt,
              loss = "mean_absolute_error",
              metrics=["mae"])
model1.fit(x.values,y.values,nb_epoch=1000)

Epoch 1/1000
363/363 [==============================] - 0s - loss: 26.1601 - mean_absolute_error: 26.1601     
Epoch 2/1000
363/363 [==============================] - 0s - loss: 11.7157 - mean_absolute_error: 11.7157     
Epoch 3/1000
363/363 [==============================] - 0s - loss: 8.3484 - mean_absolute_error: 8.3484     
Epoch 4/1000
363/363 [==============================] - 0s - loss: 7.8858 - mean_absolute_error: 7.8858     
Epoch 5/1000
363/363 [==============================] - 0s - loss: 7.6253 - mean_absolute_error: 7.6253     
Epoch 6/1000
363/363 [==============================] - 0s - loss: 7.3192 - mean_absolute_error: 7.3192     
Epoch 7/1000
363/363 [==============================] - 0s - loss: 7.3055 - mean_absolute_error: 7.3055     
Epoch 8/1000
363/363 [==============================] - 0s - loss: 7.3904 - mean_absolute_error: 7.3904     
Epoch 9/1000
363/363 [==============================] - 0s - loss: 7.3784 - mean_absolute_error: 7.3784     
Epoch 10/1000
3

363/363 [==============================] - 0s - loss: 2.4756 - mean_absolute_error: 2.4756     
Epoch 76/1000
363/363 [==============================] - 0s - loss: 2.3764 - mean_absolute_error: 2.3764     
Epoch 77/1000
363/363 [==============================] - 0s - loss: 2.4925 - mean_absolute_error: 2.4925     
Epoch 78/1000
363/363 [==============================] - 0s - loss: 2.5202 - mean_absolute_error: 2.5202     
Epoch 79/1000
363/363 [==============================] - 0s - loss: 2.7592 - mean_absolute_error: 2.7592     
Epoch 80/1000
363/363 [==============================] - 0s - loss: 2.4175 - mean_absolute_error: 2.4175     
Epoch 81/1000
363/363 [==============================] - 0s - loss: 2.4753 - mean_absolute_error: 2.4753     
Epoch 82/1000
363/363 [==============================] - 0s - loss: 2.5329 - mean_absolute_error: 2.5329     
Epoch 83/1000
363/363 [==============================] - 0s - loss: 2.4297 - mean_absolute_error: 2.4297     
Epoch 84/1000
363/363 [=

363/363 [==============================] - 0s - loss: 1.2479 - mean_absolute_error: 1.2479     
Epoch 149/1000
363/363 [==============================] - 0s - loss: 1.1544 - mean_absolute_error: 1.1544     
Epoch 150/1000
363/363 [==============================] - 0s - loss: 1.2804 - mean_absolute_error: 1.2804     
Epoch 151/1000
363/363 [==============================] - 0s - loss: 1.2028 - mean_absolute_error: 1.2028     
Epoch 152/1000
363/363 [==============================] - 0s - loss: 1.2244 - mean_absolute_error: 1.2244     
Epoch 153/1000
363/363 [==============================] - 0s - loss: 1.1190 - mean_absolute_error: 1.1190     
Epoch 154/1000
363/363 [==============================] - 0s - loss: 1.0562 - mean_absolute_error: 1.0562     
Epoch 155/1000
363/363 [==============================] - 0s - loss: 1.2013 - mean_absolute_error: 1.2013     
Epoch 156/1000
363/363 [==============================] - 0s - loss: 1.0780 - mean_absolute_error: 1.0780     
Epoch 157/1000
3

363/363 [==============================] - 0s - loss: 0.9590 - mean_absolute_error: 0.9590     
Epoch 222/1000
363/363 [==============================] - 0s - loss: 1.0187 - mean_absolute_error: 1.0187     
Epoch 223/1000
363/363 [==============================] - 0s - loss: 1.0290 - mean_absolute_error: 1.0290     
Epoch 224/1000
363/363 [==============================] - 0s - loss: 1.1883 - mean_absolute_error: 1.1883     
Epoch 225/1000
363/363 [==============================] - 0s - loss: 1.0735 - mean_absolute_error: 1.0735     
Epoch 226/1000
363/363 [==============================] - 0s - loss: 1.0063 - mean_absolute_error: 1.0063     
Epoch 227/1000
363/363 [==============================] - 0s - loss: 0.9847 - mean_absolute_error: 0.9847     
Epoch 228/1000
363/363 [==============================] - 0s - loss: 1.0564 - mean_absolute_error: 1.0564     
Epoch 229/1000
363/363 [==============================] - 0s - loss: 0.9320 - mean_absolute_error: 0.9320     
Epoch 230/1000
3

363/363 [==============================] - 0s - loss: 0.6592 - mean_absolute_error: 0.6592     
Epoch 295/1000
363/363 [==============================] - 0s - loss: 0.5653 - mean_absolute_error: 0.5653     
Epoch 296/1000
363/363 [==============================] - 0s - loss: 0.6540 - mean_absolute_error: 0.6540     
Epoch 297/1000
363/363 [==============================] - 0s - loss: 0.5421 - mean_absolute_error: 0.5421     
Epoch 298/1000
363/363 [==============================] - 0s - loss: 0.6476 - mean_absolute_error: 0.6476     
Epoch 299/1000
363/363 [==============================] - 0s - loss: 0.7370 - mean_absolute_error: 0.7370     
Epoch 300/1000
363/363 [==============================] - 0s - loss: 0.8554 - mean_absolute_error: 0.8554     
Epoch 301/1000
363/363 [==============================] - 0s - loss: 1.0654 - mean_absolute_error: 1.0654     
Epoch 302/1000
363/363 [==============================] - 0s - loss: 0.8171 - mean_absolute_error: 0.8171     
Epoch 303/1000
3

...

363/363 [==============================] - 0s - loss: 0.3896 - mean_absolute_error: 0.3896     
Epoch 660/1000
363/363 [==============================] - 0s - loss: 0.3911 - mean_absolute_error: 0.3911     
Epoch 661/1000
363/363 [==============================] - 0s - loss: 0.3768 - mean_absolute_error: 0.3768     
Epoch 662/1000
363/363 [==============================] - 0s - loss: 0.3849 - mean_absolute_error: 0.3849     
Epoch 663/1000
363/363 [==============================] - 0s - loss: 0.4960 - mean_absolute_error: 0.4960     
Epoch 664/1000
363/363 [==============================] - 0s - loss: 0.4898 - mean_absolute_error: 0.4898     
Epoch 665/1000
363/363 [==============================] - 0s - loss: 0.4935 - mean_absolute_error: 0.4935     
Epoch 666/1000
363/363 [==============================] - 0s - loss: 0.5520 - mean_absolute_error: 0.5520     
Epoch 667/1000
363/363 [==============================] - 0s - loss: 0.4452 - mean_absolute_error: 0.4452     
Epoch 668/1000
3

363/363 [==============================] - 0s - loss: 0.3366 - mean_absolute_error: 0.3366     
Epoch 733/1000
363/363 [==============================] - 0s - loss: 0.3785 - mean_absolute_error: 0.3785     
Epoch 734/1000
363/363 [==============================] - 0s - loss: 0.3919 - mean_absolute_error: 0.3919     
Epoch 735/1000
363/363 [==============================] - 0s - loss: 0.3404 - mean_absolute_error: 0.3404     
Epoch 736/1000
363/363 [==============================] - 0s - loss: 0.3226 - mean_absolute_error: 0.3226     
Epoch 737/1000
363/363 [==============================] - 0s - loss: 0.3514 - mean_absolute_error: 0.3514     
Epoch 738/1000
363/363 [==============================] - 0s - loss: 0.3227 - mean_absolute_error: 0.3227     
Epoch 739/1000
363/363 [==============================] - 0s - loss: 0.3204 - mean_absolute_error: 0.3204     
Epoch 740/1000
363/363 [==============================] - 0s - loss: 0.3978 - mean_absolute_error: 0.3978     
Epoch 741/1000
3

363/363 [==============================] - 0s - loss: 0.2591 - mean_absolute_error: 0.2591     
Epoch 806/1000
363/363 [==============================] - 0s - loss: 0.3097 - mean_absolute_error: 0.3097     
Epoch 807/1000
363/363 [==============================] - 0s - loss: 0.3482 - mean_absolute_error: 0.3482     
Epoch 808/1000
363/363 [==============================] - 0s - loss: 0.3387 - mean_absolute_error: 0.3387     
Epoch 809/1000
363/363 [==============================] - 0s - loss: 0.3602 - mean_absolute_error: 0.3602     
Epoch 810/1000
363/363 [==============================] - 0s - loss: 0.4373 - mean_absolute_error: 0.4373     
Epoch 811/1000
363/363 [==============================] - 0s - loss: 0.5000 - mean_absolute_error: 0.5000     
Epoch 812/1000
363/363 [==============================] - 0s - loss: 0.4161 - mean_absolute_error: 0.4161     
Epoch 813/1000
363/363 [==============================] - 0s - loss: 0.3709 - mean_absolute_error: 0.3709     
Epoch 814/1000
3

363/363 [==============================] - 0s - loss: 0.4839 - mean_absolute_error: 0.4839     
Epoch 878/1000
363/363 [==============================] - 0s - loss: 0.4346 - mean_absolute_error: 0.4346     
Epoch 879/1000
363/363 [==============================] - 0s - loss: 0.4440 - mean_absolute_error: 0.4440     
Epoch 880/1000
363/363 [==============================] - 0s - loss: 0.3500 - mean_absolute_error: 0.3500     
Epoch 881/1000
363/363 [==============================] - 0s - loss: 0.3642 - mean_absolute_error: 0.3642     
Epoch 882/1000
363/363 [==============================] - 0s - loss: 0.3014 - mean_absolute_error: 0.3014     
Epoch 883/1000
363/363 [==============================] - 0s - loss: 0.3085 - mean_absolute_error: 0.3085     
Epoch 884/1000
363/363 [==============================] - 0s - loss: 0.2775 - mean_absolute_error: 0.2775     
Epoch 885/1000
363/363 [==============================] - 0s - loss: 0.3550 - mean_absolute_error: 0.3550     
Epoch 886/1000
3

363/363 [==============================] - 0s - loss: 0.2285 - mean_absolute_error: 0.2285     
Epoch 951/1000
363/363 [==============================] - 0s - loss: 0.2245 - mean_absolute_error: 0.2245     
Epoch 952/1000
363/363 [==============================] - 0s - loss: 0.2883 - mean_absolute_error: 0.2883     
Epoch 953/1000
363/363 [==============================] - 0s - loss: 0.2648 - mean_absolute_error: 0.2648     
Epoch 954/1000
363/363 [==============================] - 0s - loss: 0.3264 - mean_absolute_error: 0.3264     
Epoch 955/1000
363/363 [==============================] - 0s - loss: 0.3172 - mean_absolute_error: 0.3172     
Epoch 956/1000
363/363 [==============================] - 0s - loss: 0.3274 - mean_absolute_error: 0.3274     
Epoch 957/1000
363/363 [==============================] - 0s - loss: 0.2778 - mean_absolute_error: 0.2778     
Epoch 958/1000
363/363 [==============================] - 0s - loss: 0.2774 - mean_absolute_error: 0.2774     
Epoch 959/1000
3

In [36]:
df_h_t = test_df.drop(['datetime'],axis=1)


pred_df = df_h_t

pred_df["y"] = model1.predict(pred_df.values)
pred_df["y"] = (1 - pred_df['close']) * (pred_df.y + 6)
submission1 = pd.concat([test_df_datetime, pred_df.y], axis=1)
submission1.to_csv("submission_1031_5.csv", index=False,header=False)

In [37]:
submission1

,datetime,y
0,2016-04-01,74.321625
1,2016-04-02,77.453026
2,2016-04-03,80.561310
3,2016-04-04,73.706192
4,2016-04-05,62.152229
5,2016-04-06,58.370789
6,2016-04-07,49.417297
7,2016-04-08,58.882423
8,2016-04-09,68.235329
9,2016-04-10,71.587845


In [40]:
(submission1.y + submission.y)/2

0       72.163170
1       82.426353
2       74.963364
3       74.068092
4       63.863750
5       58.030540
6       54.112589
7       58.471689
8       67.442959
9       71.311394
10      49.749622
11      56.325392
12      53.278147
13      44.965715
14      59.075583
15      58.379026
16      46.395693
17      47.285244
18      42.952003
19      44.169930
20      43.912739
21      58.081125
22      63.721859
23      50.756338
24      61.334616
25      57.150646
26      63.894217
27      61.253313
28      80.027336
29      74.138378
          ...    
335     66.447741
336     74.293144
337     86.563869
338     67.844597
339     63.326115
340     61.470026
341     64.714085
342     68.381950
343     78.331551
344     81.438118
345     80.412582
346     78.715141
347     77.885246
348     81.953102
349     93.905289
350     95.373936
351    103.595570
352     91.547386
353    106.793018
354     92.290104
355    107.994118
356    109.701256
357    103.493786
358     99.393650
359    101